In [1]:
from vanna.openai import OpenAI_Chat
from vanna.chromadb import ChromaDB_VectorStore
import os
from dotenv import load_dotenv
from langfuse.openai import openai

load_dotenv()

# Configuration des variables d'environnement pour Langfuse
os.environ["LANGFUSE_SECRET_KEY"] = os.getenv('LANGFUSE_SECRET_KEY', '')
os.environ["LANGFUSE_PUBLIC_KEY"] = os.getenv('LANGFUSE_PUBLIC_KEY', '')
os.environ["LANGFUSE_HOST"] = os.getenv('LANGFUSE_HOST', 'https://cloud.langfuse.com')

class MyVanna(ChromaDB_VectorStore, OpenAI_Chat):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        OpenAI_Chat.__init__(self, config=config)
        
        # Remplacer le client OpenAI par la version instrumentée Langfuse
        if hasattr(self, '_client'):
            # Sauvegarder la clé API
            api_key = getattr(self._client, 'api_key', config.get('api_key'))
            # Remplacer par le client Langfuse (drop-in replacement)
            self._client = openai.OpenAI(api_key=api_key)
            print("✅ Client OpenAI instrumenté avec Langfuse")

vn = MyVanna(config={'api_key':os.getenv('OPENAI_API_KEY'), 'model': 'gpt-4o-mini'})
# vn.remove_training_data()

vn.connect_to_postgres(host=os.getenv('POSTGRES_HOST'), dbname=os.getenv('POSTGRES_DB'), user=os.getenv('POSTGRES_USER'), password=os.getenv('POSTGRES_PASSWORD'), port=os.getenv('POSTGRES_PORT'))

In [2]:
from vanna.flask import VannaFlaskApp

# Créer l'application Flask avec monitoring automatique Langfuse
flask_app = VannaFlaskApp(vn, allow_llm_to_see_data=True)

print("🔍 Langfuse monitoring activé!")
print(f"📊 Dashboard: \nhttps://cloud.langfuse.com \n")
print("🚀 Toutes les requêtes OpenAI seront automatiquement trackées...")

# Vérifier la connexion Langfuse
try:
    # Test simple pour vérifier la configuration
    test_keys = [
        ("LANGFUSE_SECRET_KEY", os.getenv('LANGFUSE_SECRET_KEY')),
        ("LANGFUSE_PUBLIC_KEY", os.getenv('LANGFUSE_PUBLIC_KEY'))
    ]
    
    missing_keys = [name for name, value in test_keys if not value]
    
    if missing_keys:
        print(f"⚠️ Variables manquantes: {', '.join(missing_keys)}")
        print("💡 Ajoutez-les à votre fichier .env pour activer le monitoring")
    else:
        print("✅ Configuration Langfuse détectée")
        
except Exception as e:
    print(f"⚠️ Erreur de configuration Langfuse: {e}")

# Lancer l'application
flask_app.run()

🔍 Langfuse monitoring activé!
📊 Dashboard: 
https://cloud.langfuse.com 

🚀 Toutes les requêtes OpenAI seront automatiquement trackées...
✅ Configuration Langfuse détectée
Your app is running at:
http://localhost:8084
 * Serving Flask app 'vanna.flask'
 * Debug mode: on


Add of existing embedding ID: 6ad00744-f6f1-5fda-9243-b2de39cd6989-sql
Add of existing embedding ID: 25e202bb-2fcc-55f2-8a59-f4b872d71187-sql
Add of existing embedding ID: 352c3c4f-4364-57e0-a468-43d3b72c246b-sql
Add of existing embedding ID: 8b9021c1-0aa4-5cf1-8b0a-e0bcedc10a21-sql
Add of existing embedding ID: 6a580a8e-dc98-5e55-bcdf-e43c130baa0e-sql
Add of existing embedding ID: a2d56652-10b8-57b4-afdd-86aae9060c55-sql
Add of existing embedding ID: 7f41cbc3-77d9-5b7c-b4c0-e21509702257-sql
Add of existing embedding ID: a3a8a2c5-a0ae-5652-af1a-6459b89bc74d-sql
Add of existing embedding ID: f8b8dc1a-6cb1-5032-88c1-d84c16d370af-sql


Using model gpt-4o-mini for 4507.5 tokens (approx)
Using model gpt-4o-mini for 872.5 tokens (approx)
Using model gpt-4o-mini for 4507.5 tokens (approx)
Using model gpt-4o-mini for 872.5 tokens (approx)
